In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import time

from math import sqrt
from numpy import loadtxt
from itertools import product
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import  TfidfVectorizer


kernel_with_output = False


In [3]:
kernel_with_output = True

if kernel_with_output:
    sales_train = pd.read_csv('../input/sales_train.csv')
    items = pd.read_csv('../input/items.csv')


In [4]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [5]:
    grid = []
    for block_num in sales_train['date_block_num'].unique():
        cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales_train[sales_train['date_block_num'] == block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))


'[array([[   59, 22154,     0],\n       [   59,  2552,     0],\n       [   59,  2554,     0],\n       ...,\n       [   45,   628,     0],\n       [   45,   631,     0],\n       [   45,   621,     0]], dtype=int32), array([[   50,  3880,     1],\n       [   50,  4128,     1],\n       [   50,  4124,     1],\n       ...,\n       [   28, 12885,     1],\n       [   28, 12791,     1],\n       [   28, 13433,     1]], dtype=int32), array([[    5, 20175,     2],\n       [    5, 20083,     2],\n       [    5,    31,     2],\n       ...,\n       [    4, 12388,     2],\n       [    4, 12340,     2],\n       [    4, 10649,     2]], dtype=int32), array([[   25,  8092,     3],\n       [   25,  7850,     3],\n       [   25,  8051,     3],\n       ...,\n       [   41, 14063,     3],\n       [   41, 20690,     3],\n       [   41, 19235,     3]], dtype=int32), array([[   59, 22114,     4],\n       [   59, 20239,     4],\n       [   59, 20238,     4],\n       ...,\n       [    6,  1924,     4],\n       [ 